In [1]:
from ast import While
import pandas as pd
# Para separar datos de entrenamiento y prueba
from sklearn.model_selection import train_test_split
# Librería para SVM (Support vector machine) de clasificación
from sklearn.svm import SVC  # SVC: soporte vectorial para clasificar
# Medición de precisión
from sklearn.metrics import accuracy_score, confusion_matrix
# Generar gráficos
import matplotlib.pyplot as plt
# Operaciones numéricas
import numpy as np
# Para manipular Excel
import xlwings as xw


In [2]:
df = pd.read_excel(
    'Datos/modelo validación.xlsx',
    sheet_name='datoPruebaMayoJunio'
)

df.head()


,epa,motivo,comuna,visitas,cliente,cruce
0,31,1,9,3,7,1
1,12,3,9,3,7,0
2,21,1,3,2,11,0
3,8,1,44,2,1,0
4,19,3,43,2,1,1


In [3]:
# Viendo el equilibro de las etiquetas.
df.cruce.value_counts()


0    23612
1     3728
Name: cruce, dtype: int64

In [4]:
# Filtra los datos
df_0 = df[df['cruce'] == 0]
df_1 = df[df['cruce'] == 1]


In [5]:
# Ahora en conjunto de etiquetas 0 tiene la misma forma que las etiquetas con 1 (2131, 5)
df_0_reduced = df_0.sample(3728)
df_0_reduced.shape


(3728, 6)

In [6]:
# Uniendo los conjunto a nivel de filas. Se puede ver que quedan amontonados las equiquetas
# con 0 n la parte superior de la tabla.
df_reduced = pd.concat(
    [df_0_reduced, df_1],
    axis=0
)
df_reduced.head()


,epa,motivo,comuna,visitas,cliente,cruce
23933,8,21,24,1,2,0
16944,1,1,64,2,1,0
9210,7,1,10,1,1,0
11655,13,1,83,3,1,0
1854,10,1,55,2,1,0


In [7]:
# Con lo anterior dicho, lo que conviene es revolver los datos.
df_reduced = df_reduced.sample(frac=1)
df_reduced.head()


,epa,motivo,comuna,visitas,cliente,cruce
874,11,1,71,1,1,1
4982,15,1,6,1,1,1
11918,6,1,29,2,1,0
14471,1,15,1,1,1,1
26253,41,3,12,1,15,1


In [8]:
# Data equilibrada
df_reduced.cruce.value_counts()


1    3728
0    3728
Name: cruce, dtype: int64

In [9]:
# input
X = df_reduced[
    [
        'epa',
        'motivo',
        # 'comuna',
        'visitas',
        'cliente'
    ]
]

# Output
y = df_reduced['cruce']


In [10]:
X, y


(       epa  motivo  visitas  cliente
 874     11       1        1        1
 4982    15       1        1        1
 11918    6       1        2        1
 14471    1      15        1        1
 26253   41       3        1       15
 ...    ...     ...      ...      ...
 15620   11       1        2        1
 10998    5       1        2        1
 1024     1      15        2        1
 7841    16       3        2        1
 7152    21      15        1        1
 
 [7456 rows x 4 columns],
 874      1
 4982     1
 11918    0
 14471    1
 26253    1
         ..
 15620    1
 10998    0
 1024     1
 7841     1
 7152     1
 Name: cruce, Length: 7456, dtype: int64)

In [13]:
clf = SVC(
    C=1.0,
    kernel='rbf',
    degree=100,
    gamma='auto',
    verbose=True,
    decision_function_shape='ovr'
)


In [14]:
clf.fit(
    X.values,
    y.values.ravel(),
)


[LibSVM]

SVC(degree=100, gamma='auto', verbose=True)

In [15]:
# DATA para hacer predicciones
data = pd.read_excel(
    'Datos/modelo validación.xlsx',
    sheet_name='datosPrediccion',
).loc[:, ['EPA', 'MOTIVO', 'VISITAS', 'CLIENTE']]


data.head(2)


,EPA,MOTIVO,VISITAS,CLIENTE
0,1,3,1,1
1,1,15,1,1


In [17]:
# PREDICCION
prediccion = clf.predict(data)


c:\Users\arnif\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


In [19]:
columnaPrediccion = pd.DataFrame(prediccion)
columnaPrediccion.columns = ['Predicción']
columnaPrediccion.head()


,Predicción
0,1
1,1
2,0
3,0
4,1


In [20]:
wb = xw.Book(
    'RUTA/modelo validación.xlsx'
)
# Evita mostrar la actualizacion del archivo cada vez que se hace
wb.app.screen_updating = False
# Para hacer el archivo invisible
wb.app.visible = False

wb.sheets['datosPrediccion'].range(2, 23).value = columnaPrediccion.values
wb.save()
wb.close()
